# Frontend Frameworks & Architecture Overview

A concise guide to modern frontend frameworks (React, Vue, Angular), component patterns, state management, and rendering strategies.

```
┌─────────────────────────────────────────────────────────────────┐
│                    FRONTEND ARCHITECTURE                        │
├─────────────────┬─────────────────┬─────────────────────────────┤
│    FRAMEWORKS   │    PATTERNS     │      RENDERING              │
│  ┌───────────┐  │  ┌───────────┐  │  ┌───────────────────────┐  │
│  │  React    │  │  │Smart/Dumb │  │  │ SSR │ CSR │ SSG │ ISR │  │
│  │  Vue      │  │  │Components │  │  └───────────────────────┘  │
│  │  Angular  │  │  └───────────┘  │                             │
│  └───────────┘  │  ┌───────────┐  │  ┌───────────────────────┐  │
│                 │  │   State   │  │  │    Hydration Flow     │  │
│                 │  │Management │  │  │  Server → Client      │  │
│                 │  └───────────┘  │  └───────────────────────┘  │
└─────────────────┴─────────────────┴─────────────────────────────┘
```

## 1. React: Components, Hooks & Virtual DOM

### Core Concepts

```javascript
// Functional Component with Hooks
function Counter() {
  const [count, setCount] = useState(0);        // State Hook
  const prevCount = useRef(count);              // Ref Hook
  
  useEffect(() => {                             // Effect Hook
    document.title = `Count: ${count}`;
    prevCount.current = count;
  }, [count]);  // Dependency array
  
  const increment = useCallback(() => {         // Memoized callback
    setCount(c => c + 1);
  }, []);
  
  return <button onClick={increment}>{count}</button>;
}
```

### Virtual DOM Reconciliation

```
   State Change                 Diffing                    DOM Update
       │                           │                           │
       ▼                           ▼                           ▼
┌─────────────┐    ┌─────────────────────────────┐    ┌─────────────┐
│ New Virtual │    │  Compare Old vs New VDOM    │    │  Minimal    │
│    DOM      │───▶│  ┌───┐      ┌───┐          │───▶│  Patches    │
│   Created   │    │  │OLD│ diff │NEW│          │    │  Applied    │
└─────────────┘    │  └───┘      └───┘          │    └─────────────┘
                   └─────────────────────────────┘
                   
Key Optimizations:
• Keys for list reconciliation    • shouldComponentUpdate / React.memo
• Fiber architecture (async)      • Concurrent rendering (React 18+)
```

### Essential Hooks Reference

| Hook | Purpose | When to Use |
|------|---------|-------------|
| `useState` | Local state | Simple component state |
| `useEffect` | Side effects | API calls, subscriptions, DOM |
| `useContext` | Context consumption | Avoid prop drilling |
| `useReducer` | Complex state logic | Multiple sub-values, actions |
| `useMemo` | Memoize values | Expensive computations |
| `useCallback` | Memoize functions | Stable references for children |
| `useRef` | Mutable refs | DOM refs, previous values |

## 2. Vue: Options API vs Composition API

### Options API (Vue 2 Style)
```javascript
export default {
  data() {
    return { count: 0 };
  },
  computed: {
    doubled() { return this.count * 2; }
  },
  methods: {
    increment() { this.count++; }
  },
  mounted() {
    console.log('Component mounted');
  }
}
```

### Composition API (Vue 3)
```javascript
import { ref, computed, onMounted } from 'vue';

export default {
  setup() {
    const count = ref(0);                       // Reactive ref
    const doubled = computed(() => count.value * 2);
    const increment = () => count.value++;
    
    onMounted(() => console.log('Mounted'));
    
    return { count, doubled, increment };
  }
}

// Or with <script setup> (recommended)
<script setup>
const count = ref(0);
const doubled = computed(() => count.value * 2);
</script>
```

### Comparison

| Aspect | Options API | Composition API |
|--------|------------|------------------|
| **Organization** | By option type (data, methods) | By logical concern |
| **Reusability** | Mixins (problematic) | Composables (clean) |
| **TypeScript** | Limited support | Excellent support |
| **Learning Curve** | Easier for beginners | Requires understanding reactivity |
| **Code Splitting** | Harder | Natural by feature |

## 3. Angular: Modules, Services & Dependency Injection

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────┐
│                       NgModule                              │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────────────┐  │
│  │ Components  │  │  Directives │  │       Pipes         │  │
│  │ declarations│  │             │  │                     │  │
│  └─────────────┘  └─────────────┘  └─────────────────────┘  │
│  ┌─────────────────────────────────────────────────────┐    │
│  │              Services (providers)                    │    │
│  │    ┌──────────┐   ┌──────────┐   ┌──────────┐       │    │
│  │    │ DataSvc  │   │ AuthSvc  │   │ HttpSvc  │       │    │
│  │    └────┬─────┘   └────┬─────┘   └────┬─────┘       │    │
│  │         └──────────────┴──────────────┘              │    │
│  │                Dependency Injection                  │    │
│  └─────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────┘
```

### Core Concepts

```typescript
// Service with DI
@Injectable({ providedIn: 'root' })  // Singleton at root level
export class UserService {
  constructor(private http: HttpClient) {}  // DI injection
  
  getUsers(): Observable<User[]> {
    return this.http.get<User[]>('/api/users');
  }
}

// Component using the service
@Component({
  selector: 'app-user-list',
  template: `<li *ngFor="let user of users$ | async">{{user.name}}</li>`
})
export class UserListComponent {
  users$ = this.userService.getUsers();  // Observable stream
  
  constructor(private userService: UserService) {}  // Injected
}
```

### Injection Scopes

| Scope | Declaration | Lifetime |
|-------|-------------|----------|
| **Root** | `providedIn: 'root'` | App singleton |
| **Module** | `providers: []` in NgModule | Module singleton |
| **Component** | `providers: []` in Component | Per component instance |

## 4. Component Architecture: Smart vs Dumb (Presentational)

```
┌──────────────────────────────────────────────────────────────────┐
│                    COMPONENT HIERARCHY                           │
│                                                                  │
│    ┌─────────────────────────────────────────────────────┐      │
│    │              SMART (Container) Component            │      │
│    │  • Connects to store/services                       │      │
│    │  • Handles business logic                           │      │
│    │  • Passes data down via props                       │      │
│    └──────────────────────┬──────────────────────────────┘      │
│                           │                                      │
│              ┌────────────┴────────────┐                        │
│              ▼                         ▼                        │
│    ┌─────────────────┐       ┌─────────────────┐                │
│    │  DUMB Component │       │  DUMB Component │                │
│    │  • Pure UI only │       │  • Receives     │                │
│    │  • Receives     │       │    props        │                │
│    │    props        │       │  • Emits events │                │
│    │  • Emits events │       │  • Reusable     │                │
│    └─────────────────┘       └─────────────────┘                │
│                                                                  │
└──────────────────────────────────────────────────────────────────┘
```

### Pattern Comparison

| Aspect | Smart (Container) | Dumb (Presentational) |
|--------|-------------------|------------------------|
| **State** | Manages state, connects to store | Stateless or local UI state only |
| **Data Flow** | Fetches data, passes down | Receives via props/inputs |
| **Side Effects** | API calls, subscriptions | None (pure rendering) |
| **Reusability** | Low (app-specific) | High (generic) |
| **Testing** | Integration tests | Easy unit tests |
| **Examples** | `UserPage`, `DashboardContainer` | `Button`, `Card`, `UserAvatar` |

### Modern Variations

```
Hooks/Composables Era:
┌────────────────────────────────────────────────────┐
│  Logic extracted into reusable hooks/composables   │
│                                                    │
│  useAuth()  ──┐                                    │
│  useCart()  ──┼──▶  Component (any type)           │
│  useTheme() ──┘                                    │
│                                                    │
│  Boundary between smart/dumb becomes flexible      │
└────────────────────────────────────────────────────┘
```

## 5. State Management Approaches

```
┌─────────────────────────────────────────────────────────────────────┐
│                    STATE MANAGEMENT SPECTRUM                        │
├─────────────────────────────────────────────────────────────────────┤
│  Simple                                               Complex       │
│    │                                                      │         │
│    ▼                                                      ▼         │
│  Local    Props     Context/    Stores      Flux/Redux   Server    │
│  State    Drilling  Provide     (Zustand,   (Redux,      State     │
│                     Inject      Pinia)      NgRx)        (TanStack)│
│                                                                     │
│  useState  ─────▶  useContext ─────▶  Zustand ─────▶  Redux       │
│  ref()     ─────▶  provide()   ─────▶  Pinia   ─────▶  TanStack   │
│  signals   ─────▶  services    ─────▶  NgRx                        │
└─────────────────────────────────────────────────────────────────────┘
```

### Comparison Matrix

| Solution | Framework | Pattern | Complexity | Best For |
|----------|-----------|---------|------------|----------|
| **useState/ref** | React/Vue | Local | Low | Component state |
| **Context/Provide** | React/Vue | DI | Low-Med | Theme, auth, i18n |
| **Zustand** | React | Proxy | Low | Simple global state |
| **Pinia** | Vue | Stores | Low-Med | Vue apps |
| **Redux/RTK** | React | Flux | Medium | Large apps, time-travel |
| **NgRx** | Angular | Redux | High | Enterprise Angular |
| **TanStack Query** | Any | Server | Medium | Async/cached data |
| **Signals** | Angular/Solid | Reactive | Low | Fine-grained reactivity |

### Redux Flow

```
┌─────────┐    dispatch    ┌─────────┐    pure fn    ┌─────────┐
│  View   │───────────────▶│ Action  │──────────────▶│ Reducer │
└────▲────┘                └─────────┘               └────┬────┘
     │                                                    │
     │              ┌─────────┐                          │
     └──────────────│  Store  │◀─────────────────────────┘
       subscribe    └─────────┘    new state
```

## 6. Rendering Strategies: SSR vs CSR vs SSG vs ISR

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      RENDERING STRATEGIES                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  CSR (Client-Side)           SSR (Server-Side)                         │
│  ┌─────────────────┐         ┌─────────────────┐                       │
│  │ Browser Request │         │ Browser Request │                       │
│  └────────┬────────┘         └────────┬────────┘                       │
│           ▼                           ▼                                 │
│  ┌─────────────────┐         ┌─────────────────┐                       │
│  │  Empty HTML +   │         │  Server renders │                       │
│  │  JS Bundle      │         │  full HTML      │                       │
│  └────────┬────────┘         └────────┬────────┘                       │
│           ▼                           ▼                                 │
│  ┌─────────────────┐         ┌─────────────────┐                       │
│  │  JS runs,       │         │  HTML displayed │                       │
│  │  renders DOM    │         │  + hydration    │                       │
│  └─────────────────┘         └─────────────────┘                       │
│                                                                         │
│  SSG (Static Generation)     ISR (Incremental Static)                  │
│  ┌─────────────────┐         ┌─────────────────┐                       │
│  │  BUILD TIME     │         │  BUILD + Runtime│                       │
│  │  HTML generated │         │  Revalidation   │                       │
│  └────────┬────────┘         └────────┬────────┘                       │
│           ▼                           ▼                                 │
│  ┌─────────────────┐         ┌─────────────────┐                       │
│  │  CDN serves     │         │  Stale-while-   │                       │
│  │  static files   │         │  revalidate     │                       │
│  └─────────────────┘         └─────────────────┘                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Strategy Comparison

| Strategy | Rendering Time | SEO | TTFB | Use Cases |
|----------|---------------|-----|------|----------|
| **CSR** | Client (runtime) | Poor* | Fast | SPAs, dashboards, auth'd content |
| **SSR** | Server (per request) | Excellent | Slower | Dynamic personalized pages |
| **SSG** | Build time | Excellent | Fastest | Blogs, docs, marketing |
| **ISR** | Build + revalidate | Excellent | Fast | E-commerce, news sites |

*Can be improved with prerendering or dynamic rendering for bots

### Framework Support

| Framework | CSR | SSR | SSG | ISR |
|-----------|-----|-----|-----|-----|
| **Next.js** | ✅ | ✅ | ✅ | ✅ |
| **Nuxt** | ✅ | ✅ | ✅ | ✅ |
| **Angular Universal** | ✅ | ✅ | ✅ | ⚠️ |
| **Astro** | ✅ | ✅ | ✅ | ✅ |
| **SvelteKit** | ✅ | ✅ | ✅ | ✅ |

## 7. Framework Comparison Summary

| Aspect | React | Vue | Angular |
|--------|-------|-----|---------|
| **Type** | Library | Framework | Full Framework |
| **Language** | JSX/TSX | SFC (template) | TypeScript |
| **Data Binding** | One-way | Two-way (v-model) | Two-way |
| **State Reactivity** | Immutable (setState) | Proxy-based (reactive) | Zone.js / Signals |
| **Component Style** | Functions + Hooks | Options or Composition | Classes + Decorators |
| **Routing** | react-router (separate) | vue-router (official) | @angular/router (built-in) |
| **Learning Curve** | Medium | Low-Medium | High |
| **Bundle Size** | ~40KB | ~33KB | ~130KB+ |
| **Enterprise Use** | High | Growing | Very High |
| **Meta-framework** | Next.js, Remix | Nuxt | Angular Universal |

### Decision Guide

```
Choose React when:
  ├── Large ecosystem needed
  ├── Team knows JavaScript well
  └── Maximum flexibility required

Choose Vue when:
  ├── Gradual adoption needed
  ├── Simpler learning curve
  └── Template syntax preferred

Choose Angular when:
  ├── Enterprise/large team
  ├── Full framework with conventions
  └── TypeScript-first required
```

## 8. Quick Reference & Best Practices

### Performance Optimization Checklist

| Technique | React | Vue | Angular |
|-----------|-------|-----|---------|
| **Lazy Loading** | `React.lazy()` | `defineAsyncComponent` | `loadChildren` |
| **Memoization** | `React.memo`, `useMemo` | `computed` | `pure` pipe, `OnPush` |
| **Virtual Lists** | react-window | vue-virtual-scroller | cdk-virtual-scroll |
| **Code Splitting** | Dynamic imports | Dynamic imports | Lazy modules |

### Architecture Best Practices

```
✅ DO:
  • Keep components small and focused
  • Lift state only when necessary
  • Use composition over inheritance
  • Colocate related code (styles, tests, types)
  • Implement error boundaries
  • Use TypeScript for large projects

❌ AVOID:
  • Prop drilling beyond 2-3 levels
  • Business logic in components
  • Mutating state directly
  • Over-engineering state management
  • Premature optimization
```

### Folder Structure (Feature-Based)

```
src/
├── features/
│   ├── auth/
│   │   ├── components/
│   │   ├── hooks/          # or composables/ for Vue
│   │   ├── services/
│   │   └── index.ts
│   └── products/
├── shared/
│   ├── components/
│   ├── hooks/
│   └── utils/
├── app/                    # Root component, routing
└── main.ts
```